In [1]:
import numpy as np
import os


data = np.load('outputs/embedded_3d.npy')


if not os.path.exists('outputs/vectors.tsv'):
    np.savetxt("outputs/vectors.tsv", data, delimiter="\t")
    
else:
    vectors = np.loadtxt("outputs/vectors.tsv", delimiter="\t")

### Plotly


In [3]:
import plotly.express as px
import matplotlib.pyplot as plt

# Optional labels
labels = [f"{i}" for i in range(len(data))]

# Convert to DataFrame
import pandas as pd
df = pd.DataFrame(data, columns=["x", "y", "z"])
df["label"] = labels

# Create 3D scatter plot
fig = px.scatter_3d(df, x="x", y="y", z="z", text="label", hover_name="label")
fig.update_layout(
    width=800,
    height=600
)
fig.show()

### TensorBoard

In [2]:
import tensorflow as tf
import os
import numpy as np

weights = tf.Variable(vectors, name='embedding')  # Name is critical

# Create log directory
log_dir = "logs/projector"
os.makedirs(log_dir, exist_ok=True)

# Save embeddings as checkpoint
checkpoint = tf.train.Checkpoint(embedding=weights)
checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

# Save metadata
with open(os.path.join(log_dir, 'metadata.tsv'), 'w') as f:
    for i in range(len(vectors)):
        f.write(f"{i}\n")



In [9]:
# Set up projector config
from tensorboard.plugins import projector

# Set up config.
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = "metadata.tsv"

# Save config file
projector.visualize_embeddings(log_dir, config)

In [18]:
# Now run tensorboard against on log data we just saved.
%tensorboard --logdir /logs/projector

UsageError: Line magic function `%tensorboard` not found.
